Introduction

The purpose of this project is to analyze credit card data in order to identify trends that may cause low/high balances. To achieve this, we will be using clustering to seperate people into groups based on their card usage frequency, and then using linear regression to identify whether these groups tend to rach up high balances or not.

____________________________________________________________________________________________________________________________________________________

Pip Installs & Imports

In [3]:
pip install pandas numpy scikit-learn matplotlib seaborn

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score